In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets.skyline12 import Skyline12
from sklearn.model_selection import train_test_split

skyline12 = Skyline12('/storage/skyline12/data')
train_set, test_set = train_test_split(list(skyline12), train_size=100, random_state=0)
train_set, validation_set = train_test_split(train_set, train_size=0.8, random_state=42)
len(train_set), len(validation_set), len(test_set)

(80, 20, 20)

In [3]:
import tensorflow as tf
import numpy as np
from datasets.skyline12 import create_augment_fn

augment = create_augment_fn()
NUM_CLASSES = 5


def preprocess(x, y, z):
    x, y, z = augment(x, y, z)
    x = x.astype('float32')
    x /= 255.0
    y[y >= NUM_CLASSES - 1] = NUM_CLASSES - 1
    y = tf.keras.utils.to_categorical(y, num_classes=NUM_CLASSES)
    z[z > 0] = 1
    z = np.expand_dims(z, -1)
    return x, y, z


FOLDS = 10
train_ds = tf.data.Dataset.from_generator(
    lambda: (preprocess(x, y, z) for x, y, z in train_set),
    (tf.dtypes.float32, tf.dtypes.uint8, tf.dtypes.float32),
    (
        tf.TensorShape([512, 512, 3]),
        tf.TensorShape([512, 512, NUM_CLASSES]),
        tf.TensorShape([512, 512, 1])
    )
).map(lambda x, y, z: (x, [y, z])).repeat(FOLDS)
validation_ds = tf.data.Dataset.from_generator(
    lambda: (preprocess(x, y, z) for x, y, z in validation_set),
    (tf.dtypes.float32, tf.dtypes.uint8, tf.dtypes.float32),
    (
        tf.TensorShape([512, 512, 3]),
        tf.TensorShape([512, 512, NUM_CLASSES]),
        tf.TensorShape([512, 512, 1])
    )
).map(lambda x, y, z: (x, [y, z])).repeat(FOLDS // 2)

In [3]:
from models.unet import create_unet

unet = create_unet()
unet.load_weights('checkpoints/baseline-weights.h5')

In [22]:
import wandb
from wandb.keras import WandbCallback
from callbacks import WandbLogPredictions

early_stopper = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
class_labels = {
    0: 'Borders',
    1: 'Sky',
    2: 'Unknown',
    3: 'Unknown',
    4: 'Skyscrapers',
}
data_to_log = next(iter(validation_ds.batch(20)))
log_predictions = WandbLogPredictions(data_to_log, class_labels)
AUTOTUNE = tf.data.experimental.AUTOTUNE

wandb.init(project="skyline12-augmentations", tags=['baseline'])
unet.fit(
    train_ds.batch(3).cache('temp/ds_cache/').prefetch(AUTOTUNE),
    epochs=200,
    validation_data=validation_ds.batch(3).cache('temp/ds_cache/').prefetch(AUTOTUNE),
    callbacks=[
        early_stopper,
        WandbCallback(),
        log_predictions,
        tf.keras.callbacks.ModelCheckpoint('checkpoints/best-baseline', save_best_only=True, save_weights_only=True)
    ]
)

wandb: Wandb version 0.10.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/200
    267/Unknown - 123s 459ms/step - loss: 0.9504 - categorical_mean_iou_1: 0.2204 - accuracy: 0.6476WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: checkpoints/best-baseline/assets
267/267 [==============================] - 178s 666ms/step - loss: 0.9504 - categorical_mean_iou_1: 0.2204 - accuracy: 0.6476 - val_loss: 0.8463 - val_categorical_mean_iou_1: 0.2648 - val_accuracy: 0.7164
Epoch 2/200
267/267 [==============================] - 173s 647ms/step - loss: 0.7740 - categorical_mean_iou_1: 0.2734 - accuracy: 0.7117 - val

In [28]:
unet.load_weights('checkpoints/baseline-weights.h5')